In [1]:
!apt-get update
!apt-get install -y tesseract-ocr tesseract-ocr-ben poppler-utils
!pip install pytesseract pdf2image pillow pandas regex


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,599 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,966 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.2 kB

In [4]:
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())


Saving Agriculure Education Class 8.pdf to Agriculure Education Class 8.pdf
dict_keys(['Agriculure Education Class 8.pdf'])


In [5]:
from pdf2image import convert_from_path
import pytesseract
import re

def clean_bangla_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\u0980-\u09FF0-9।, ]', '', text)
    return text.strip()

all_rows = []

for pdf_name in uploaded.keys():
    pages = convert_from_path(pdf_name, dpi=300)
    print(f"Processing {pdf_name}, pages: {len(pages)}")

    for page_no, page in enumerate(pages, start=1):
        raw_text = pytesseract.image_to_string(page, lang="ben")
        clean_text = clean_bangla_text(raw_text)

        if len(clean_text) > 20:  # ignore empty/noise pages
            all_rows.append({
                "file": pdf_name,
                "page": page_no,
                "text": clean_text
            })

print("Total extracted pages:", len(all_rows))


Processing Agriculure Education Class 8.pdf, pages: 137
Total extracted pages: 135


In [6]:
import pandas as pd

df = pd.DataFrame(all_rows)

csv_name = "extracted_bangla_text.csv"
df.to_csv(csv_name, index=False, encoding="utf-8-sig")

print("✅ CSV created:", csv_name)
df.head()


✅ CSV created: extracted_bangla_text.csv


,file,page,text
0,Agriculure Education Class 8.pdf,2,জাতীয় শিক্ষাক্রম ও পাঠ্যপুস্তক বোর্ড কর্তৃক ২...
1,Agriculure Education Class 8.pdf,3,"জাতীয় শিক্ষাক্রম ও পাঠ্যপুস্তক বোর্ড ৬৯৭০, মত..."
2,Agriculure Education Class 8.pdf,4,প্রসঙ্গ কথা বর্তমানে প্রাতিষ্ঠানিক শিক্ষার উপয...
3,Agriculure Education Class 8.pdf,6,প্রথম অধ্যায় বাংলাদেশের কৃষি ও আন্তর্জাতিক প্...
4,Agriculure Education Class 8.pdf,7,২ কৃষিশিক্ষা এ অধ্যায় শেষে আমরা কৃষিতে বিজ্ঞ...


In [7]:
!ls -lh


total 17M
-rw-r--r-- 1 root root  17M Jan  8 06:54 'Agriculure Education Class 8.pdf'
-rw-r--r-- 1 root root 467K Jan  8 07:36  extracted_bangla_text.csv
drwxr-xr-x 1 root root 4.0K Dec 11 14:34  sample_data
